In [34]:
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import glob
import json

import cv2
import torch
from tqdm import tqdm_notebook
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader
from torchvision import models

In [35]:
META_DIR = "/kaggle/input/iwildcam2022-fgvc9/metadata/metadata/"
TRAIN_DIR = "/kaggle/input/iwildcam2022-fgvc9/train/train/"

train_data = json.load(open(META_DIR + 'iwildcam2022_train_annotations.json'))
test_data = json.load(open(META_DIR + 'iwildcam2022_test_information.json'))

In [36]:
df_train = pd.DataFrame({'id': [item['id'] for item in train_data['annotations']],
                                'category_id': [item['category_id'] for item in train_data['annotations']],
                                'image_id': [item['image_id'] for item in train_data['annotations']],
                                'seq_id': [item['seq_id'] for item in train_data['images']],
                                'file_name': [item['file_name'] for item in train_data['images']],
                                'location': [item['location'] for item in train_data['images']],
                                'seq_num_frames': [item['seq_num_frames'] for item in train_data['images']],
                                'seq_frame_num': [item['seq_frame_num'] for item in train_data['images']]})



In [37]:
# %%time
# import tqdm

# indices = []
# for i in tdqm(df_train['file_name']):
#     try:
#         Image.open(TRAIN_DIR + i)
#     except:        
#         print(i)
#         df_train.drop(df_train.loc[df_train['file_name']==i].index, inplace=True)

In [38]:
seq_count = pd.read_csv(META_DIR + 'train_sequence_counts.csv')

In [39]:
seq_count.head

<bound method NDFrame.head of                                     seq_id  count
0     95dae922-21bc-11ea-a13a-137349068a90      9
1     2fff6c8a-7d42-11eb-8fb5-0242ac1c0002      9
2     300cc650-7d42-11eb-8fb5-0242ac1c0002      9
3     30125958-7d42-11eb-8fb5-0242ac1c0002      9
4     301d2d24-7d42-11eb-8fb5-0242ac1c0002      9
...                                    ...    ...
1775  9925c372-21bc-11ea-a13a-137349068a90      8
1776  9932df3a-21bc-11ea-a13a-137349068a90      8
1777  99556aa0-21bc-11ea-a13a-137349068a90      5
1778  99577458-21bc-11ea-a13a-137349068a90      4
1779  99592c3a-21bc-11ea-a13a-137349068a90      5

[1780 rows x 2 columns]>

In [40]:
import torchvision

In [41]:
import tqdm

In [42]:
import torchvision.transforms as T
import torch.nn.functional as F

In [43]:
# https://github.com/ultralytics/yolov3/blob/master/utils/datasets.py code for yolo dataset

In [44]:
transforms = T.Compose(
           [
            T.ConvertImageDtype(torch.float32),
            T.Normalize(mean=0.5, std=0.5),  # map [0, 1] into [-1, 1]
            T.Resize(size=(1024, 1024)),
           ])

class CamDataset(Dataset):
    
    def __init__(self,
                 df,
                 images_dir,
                 seq_count,
                 img_size=1024
                ):
        self.df = df
        self.images_dir = images_dir
        self.seq_count = seq_count
        self.img_size = img_size
    
    def __len__(self):
        return self.seq_count.shape[0]
    
    def __getitem__(self, idx): 
        cur_idx_row = self.seq_count.iloc[idx]
        y = int(cur_idx_row['count'])
        img_rows = self.df.loc[self.df.seq_id == cur_idx_row['seq_id']]
        out = torch.zeros(1024,1024,3,img_rows.iloc[0].seq_num_frames)
        for index, img_row in img_rows.iterrows():
           img_path = os.path.join(self.images_dir, img_row['file_name'])
           
           out[:,:,:,img_row['seq_frame_num']] = transforms(torchvision.io.read_image(img_path)).permute(1,2,0)
        padded_out = F.pad(out, pad=(10 - out.shape[3], 0, 0, 0, 0, 0, 0, 0))
        return padded_out, y    

        
            
        

In [45]:
# def collate_fn(data):
#     """
#        data: is a list of tuples with (example, label, length)
#              where 'example' is a tensor of arbitrary shape
#              and label/length are scalars
#     """
#     examples, labels = zip(*data)
#     F.pad(source, pad=(0, 0, 0, 70 - source.shape[0])

#     return features.float(), labels.long(), lengths.long()

In [46]:
from torch.utils.data import DataLoader
training_data = CamDataset(df_train, TRAIN_DIR, seq_count)
train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)


In [47]:
%%time
train_features, train_labels = next(iter(train_dataloader))

CPU times: user 30.9 s, sys: 10.4 s, total: 41.3 s
Wall time: 24.8 s


In [48]:
!pip install cython; pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-lzk00d64
  Running command git clone --filter=blob:none -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-lzk00d64
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=369323 sha256=8ef87bfc2962ef7fdcdfdc26a04d385ca0ca6a705cbc33c2fe53c5585172b9d6
  Stored in directory: /tmp/pip-ephem-wheel-cache-0zhscs9x/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.4
    Uninstalling pycocotools-2.0.4:
      Successfully uninstalled pycocotools-2.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the followi

In [49]:
!pip install -r '/kaggle/input/centertrack/requirements.txt'

  Using cached pycocotools-2.0.4-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0


In [50]:
!cd /kaggle/input/centertrack/src/lib/model/networks/
!git clone https://github.com/CharlesShang/DCNv2/ # clone if it is not automatically downloaded by `--recursive`.
!cd DCNv2
!./make.sh

fatal: destination path 'DCNv2' already exists and is not an empty directory.
/bin/bash: ./make.sh: No such file or directory


In [63]:
%tb
import sys
CENTERTRACK_PATH = '/kaggle/input/centertrack/src/lib/'
sys.path.insert(0, CENTERTRACK_PATH)

from detector import Detector
from opts import opts

MODEL_PATH = '/kaggle/input/centertrackmodel/coco_tracking.pth'
TASK = 'coco' # or 'tracking,multi_pose' for pose tracking and 'tracking,ddd' for monocular 3d tracking
print('{} --load_model {}'.format(TASK, MODEL_PATH).split(' '))
print('a')
default_dataset_info = {
      'ctdet': 'coco', 'multi_pose': 'coco_hp', 'ddd': 'nuscenes',
      'tracking,ctdet': 'coco', 'tracking,multi_pose': 'coco_hp', 
      'tracking,ddd': 'nuscenes'
    }
ops_ins = opts()
opt = ops_ins.parse('{} --load_model {}'.format(TASK, MODEL_PATH).split(' '))
print(opt)
from dataset.dataset_factory import dataset_factory
train_dataset = default_dataset_info[opt.task] if opt.task in default_dataset_info else 'coco'
dataset = dataset_factory[train_dataset]
opt = ops_ins.update_dataset_info_and_set_heads(opt, dataset)
detector = Detector(opt)



TypeError: 'NoneType' object is not callable

['coco', '--load_model', '/kaggle/input/centertrackmodel/coco_tracking.pth']
a
Fix size testing.
training chunk_sizes: [32]
Namespace(K=100, add_05=False, amodel_offset_weight=1, arch='dla_34', aug_rot=0, backbone='dla34', batch_size=32, chunk_sizes=[32], custom_dataset_ann_path='', custom_dataset_img_path='', data_dir='/kaggle/input/centertrack/src/lib/../../data', dataset='coco', dataset_version='', debug=0, debug_dir='/kaggle/input/centertrack/src/lib/../../exp/coco/default/debug', debugger_theme='white', demo='', dense_reg=1, dep_weight=1, depth_scale=1, dim_weight=1, dla_node='dcn', down_ratio=4, efficient_level=0, eval_val=False, exp_dir='/kaggle/input/centertrack/src/lib/../../exp/coco', exp_id='default', fix_res=True, fix_short=-1, flip=0.5, flip_test=False, fp_disturb=0, gpus=[0], gpus_str='0', head_conv=256, head_kernel=3, hm_disturb=0, hm_hp_weight=1, hm_weight=1, hp_weight=1, hungarian=False, ignore_loaded_cats=[], input_h=-1, input_res=-1, input_w=-1, keep_res=False, kitti

TypeError: 'NoneType' object is not callable

In [ ]:
images = ['''image read from open cv or from a video''']
for img in images:
  ret = detector.run(img)['results']